<a href="https://www.kaggle.com/code/dimasjackson/supernovae-analysis-with-mcmc?scriptVersionId=128810966" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Supernovae Analysis with Markov Chain Monte Carlo

### Abstract

In this project we will use the Pantheon+SH0ES luminosity distances data of 1701 Supernovae to estimate cosmological parameters. The parameters that we are interested in are the Hubble constant $H_0$, the matter density fraction $\Omega_m$ and the dark energy fraction $\Omega_\Lambda$, which is responsible for the accelerated expansin of the universe. The bayesian statistics will be adopted to calculate the posterior probability distribution of the parameters using the likelihood distribution and the prior probability. The likelihood for a set of cosmological parameters will be computed given the experimental supernovae distances. Also, we will implement the emcee algorithm to generate samples with a Markov Chain Monte Carlo (MCMC) model starting from the parameters values that maximize the likelihood. Finally, the results will be plotted in the space of parameters and the best values estimated avereging over some percentile of the samples.

# Installing emcee

First we will install the most recently version of [emcee](https://emcee.readthedocs.io/en/stable/) library and the package Corner for corner plots. The emcee is an MIT licensed pure-Python implementation of Goodman & Weare’s Affine Invariant Markov chain Monte Carlo (MCMC) Ensemble sampler that will be used to estimate the posterior probability function. 

In [ ]:
#Installing the EMCEE - Markov Chain Monte Carlo library
!pip install -U emcee

In [ ]:
# Installing the Corner plot module
! python -m pip install corner

In [ ]:
# Settingplots in the notebook from matplotlib
%matplotlib inline

import numpy as np # Import numpy for linear algebra computations
import matplotlib.pyplot as plt # Import Matplotlib and seabor to create plots
import seaborn as sns
import pandas as pd # Import pandas to handle the data set

In [ ]:
# Import the EnsembleSampler to generate samples of a Markov Chain and the corner plot
from emcee import EnsembleSampler
from emcee.backends import HDFBackend
from corner import corner
import sys

We will load the data of 1701 supernovae events from the [Phanteon+SH0ES Git Hub Data Release](https://github.com/PantheonPlusSH0ES/DataRelease/tree/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR) colaboration, download the data and set your directory properly.

In [ ]:
df = pd.read_table("/kaggle/input/supernovae-pantheonsh0es-data/PantheonSH0ES.dat", sep="\s+")

In [ ]:
df.head()

In [ ]:
df.shape

We are interested in the distance modulos data as a function of the redshift in the Cosmic Microwave Background (CMB) refeerence system.

In [ ]:
plt.figure(figsize=(13,7))
sns.scatterplot(x=df['zCMB'], y=df['MU_SH0ES']).set(title='Supernovae Distance Modulos per Redshift at CMB')
plt.xlabel("zCMB")
plt.ylabel("Distance Modulus")

# Defining the Theoretical Model

In [ ]:
# Defining the cosmological density parameters (just our first guess)
h_guess = 0.67
Omega_Lambda_guess, Omega_m_guess, H0_guess, = 0.69, 0.31, h_guess*100/(3.086*10**19) #H0 in 1 over secconds

It is important to note that we will estimate the parameter $h$ defined by
$$ H_0 = h\times 100 {\rm Km/sMpc}$$
because $h$ is of the same order of magnitude of $\Omega_m$ and $\Omega_\Lambda$.

Our first guess to the cosmological parameters that will be estimated is:

In [ ]:
print('Omega_Lambda_guess = {:.2f}'.format(Omega_Lambda_guess))
print('Omega_m_guess = {:.2f}'.format(Omega_m_guess))
print('h_guess = {:.2f}'.format(h_guess))

The luminosity distance in a flat expanding Friedmann-Lemaitre-Robertson-Walker university is given by:
$$ d_L = (1+z)d_H \int_{0}^{z}dz^{\prime} \left[ \Omega_r(1+z^{\prime})^4 + \Omega_m(1+z^{\prime})^3+ \Omega_{\Lambda} \right]^{-1/2} \,,$$
where $d_H=c/H_0$ is the Hubble distance.

To calculate the luminosity distance it is necessary to perform a numerical integration

In [ ]:
# Importing the numerical integration package
from scipy.integrate import quad

In [ ]:
# The function we want to integrate
def f(z, Omega_Lambda, Omega_m):
    Omega_r = 9*10**(-5)
    return 1/(np.sqrt(Omega_r*(1+z)**4+Omega_m*(1+z)**3+Omega_Lambda))

In [ ]:
# call quad to numerical integrate f using quadrature method and store the results in a list
L=[]
for z in np.arange(0.001, 2.5, 0.0014):
    res, err = quad(f, 0, z, args=(Omega_Lambda_guess, Omega_m_guess)) # I discarted the integration error, but it can be stored
    L.append(res)
len(L)

In [ ]:
cLight = 3*10**8 # Speed of light in vacuum
dH = cLight/H0_guess # Hubble length in Mpc

In [ ]:
# Calculating the luminosity distance
z_list=[]
for z in np.arange(0.001,2.5,0.0014):
    z_list.append(z)
dL=[]
for k in np.arange(0, 1785):
    dL.append(dH*(1+z_list[k])*L[k]/(3*10**22))
print(len(z_list))
print(len(dL))

What is actually measured is the distance modulos, defined by
$$ \mu = 5 [\log_{10}{(d_L)} + 5] $$

In [ ]:
# Calculating the distance modulos
mu_model=[]
for k in range(0, 1701):
    mu_model.append((5*(np.log10(np.abs(dL[k])) + 5)))
len(mu_model)

In [ ]:
# The z_list array and the distance modulus must be the same length, so we will discard the remaining points
del z_list[1701:1785]
len(z_list)

In [ ]:
# Numerical interpolation of the results
from scipy.interpolate import interp1d # Interpolation function

mu_model_inter = interp1d(z_list, mu_model, kind='cubic')

In [ ]:
# Plot the numerical interpolation function of the theoretical model
z_var = df['zCMB']
plt.figure(figsize=(13,7))
plt.plot(z_var, mu_model_inter(z_var))
plt.title("Numerical Interpolation Function of the Diatance Modulos")
plt.xlabel("zCMB")
plt.ylabel("Theoretical Distance Modulus")
plt.savefig('theo_dist.png')

In [ ]:
# Plot the theoretical experimental comparison of the distance modulos
plt.figure(figsize=(15,7))
# Plot the error bars
x_exp=df['zCMB']
y_exp=df['MU_SH0ES']
plt.errorbar(x_exp, y_exp, xerr=df['zCMBERR'], yerr=df['MU_SH0ES_ERR_DIAG'], fmt=",c", ecolor="y", capsize=0)
# Plot the true line
y_theo=mu_model_inter
plt.plot(x_exp, y_theo(x_exp), "r", alpha=0.3, lw=3)
plt.title("Theoretical and Experimental Comparison of the Distance Modulos")
legend = ("mu_the","mu_obs")
plt.legend(legend)
plt.xlabel("zCMB")
plt.ylabel("Distance Modulus")
plt.savefig('dist_modulus.png')

This is a comparison between theoretical and experimental distance modulos of 1701 supernovae. The error bars comes from the Pantheon+SH0ES data, the uncertainty on MU_SH0ES was determined from the diagonal of the covariance matrix and are only for plot purpose, cannot be used to fit cosmological parameters.

# Implementing the Model in a Pipeline

In [ ]:
def mu_model_pipe(Omega_Lambda, Omega_m, h):
    L=[]
    for z in np.arange(0.001, 2.5, 0.0014):
        res, err = quad(f, 0, z, args=(Omega_Lambda, Omega_m)) # I discarted the integration error, but it can be stored
        L.append(res)
    H0 = h*100/(3.086*10**19)
    cLight = 3*10**8 # Speed of light in vacuum
    dH = cLight/H0 # Hubble length in Mpc
    # Calculating the luminosity distance
    z_list=[]
    for z in np.arange(0.001,2.5,0.0014):
        z_list.append(z)
    dL=[]
    for k in np.arange(0, 1785):
        dL.append(dH*(1+z_list[k])*L[k]/(3*10**22))
    # Calculating the distance modulus
    mu_model=[]
    for k in range(0, 1701):
        mu_model.append((5*(np.log10(np.abs(dL[k])) + 5)))
    del z_list[1701:1785]
    mu_model_inter = interp1d(z_list, mu_model, kind='cubic')
    return mu_model_inter

In [ ]:
mu_test = mu_model_pipe(Omega_Lambda_guess, Omega_m_guess, h_guess)

In [ ]:
# Testing the mu_model_pipe function
plt.figure(figsize=(13,7))
plt.plot(z_var, mu_test(z_var))
plt.title("Testing the mu_model_pipe Function")
plt.xlabel("zCMB")
plt.ylabel("Theoretical Distance Modulus")

# Importing the Covariance Matrix

The covariance matrix will be imported from the [Pantheon+SH0ES Git Hub Data Release](https://github.com/PantheonPlusSH0ES/DataRelease/tree/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR). The format of the covariance (.cov) file is NxN lines where the matrix should be read in sequentially.  The first line gives the number of rows/columns in the matrix (N=1701).  The STATONLY matrix has only elements that correspond to the statistical distance uncertainties for individual SNe. This includes intrinsic scatter off-diagonal components when the light-curves represent the same SN observed by different surveys.

In [ ]:
cov_aux = pd.read_table('/kaggle/input/pantheon-cov-matrix/Pantheon%2BSH0ES_STAT%2BSYS.cov')
cov_aux.head()

In [ ]:
cov_aux.shape

In [ ]:
cov_matrix = [] # Read the covariance matrix and store in a array
for i in range(1701):
    row = []
    for j in range(1701*i,1701*(i+1)):
        row.append(cov_aux['1701'][j])
    cov_matrix.append(row)

In [ ]:
len(cov_matrix)

In [ ]:
cov_matrix[0][0]

In [ ]:
cov_matrix[1700][1700]==cov_aux['1701'][2893400] # Verifying if the matrix was read with succes

In [ ]:
cov_matrix = np.array(cov_matrix) # Converting the matrix list into a Numpy array format 
cov_matrix[0]

In [ ]:
cov_matrix.shape

In [ ]:
inv = np.linalg.inv(cov_matrix) # Calculating the inverse matrix
inv.shape

In [ ]:
inv[0] # Print the first line of the inverse

In [ ]:
np.linalg.eig(cov_matrix)[1] # Calculates the eigenvalues of the covariance matrix



In [ ]:
def log_likelihood(theta, mu_data, cov):
    """Compute the Gaussian log-likelihood for a given distance modulus data, distance modulus theorical and covariance."""
    Omega_Lambda, Omega_m, h = theta
    cov_inv = np.linalg.inv(cov)
    cov_eigvals = np.linalg.eig(cov)
    cov_len = len(cov)
    mu_model_array=[]
    mu_aux = mu_model_pipe(Omega_Lambda, Omega_m, h)
    for z in z_var:
        mu_model_array.append(mu_aux(z))
    mu_model_array=np.array(mu_model_array)
    return -0.5 * (np.matmul((mu_model_array - mu_data) , np.matmul(cov_inv , (mu_model_array - mu_data)))) - .5 * (np.sum(np.log(np.absolute(cov_eigvals[1]))) + cov_len * np.log(2*np.pi))

In [ ]:
# Testing the log_likelihood:
mu_data = np.array(df['MU_SH0ES'])
cov=cov_matrix
theta = (Omega_Lambda_guess, Omega_m_guess, h_guess)
log_likelihood(theta, mu_data, cov)

Now, the scipy minimize function will be used to compute the parameters $\Omega_\Lambda, \Omega_m, h$ that optimizes the likelihood for the given data. It’s worth noting that the optimize module minimizes functions whereas we would like to maximize the likelihood. This goal is equivalent to minimizing the negative likelihood (or in this case, the negative log likelihood).

In [ ]:
# Defining a random value to start the optimization
np.random.seed(62) # Generate a random number from 62
random_vec = 0.01*np.absolute(np.random.randn(3))
initial = np.array([Omega_Lambda_guess, Omega_m_guess, h_guess]) - random_vec 
initial

In [ ]:
# Define the boundary values for the parameters:
Omega_Lambda_min = 0.67
Omega_Lambda_max = 0.71
Omega_m_min = 0.29
Omega_m_max = 0.33
h_min = 0.65 
h_max = 0.69

theta_min = (Omega_Lambda_min,Omega_m_min,h_min)
theta_max = (Omega_Lambda_max,Omega_m_max,h_max)

bnds = (theta_min, theta_max)

print(bnds)

In [ ]:
# Set the bounds in a properly format
from scipy.optimize import Bounds
bd = Bounds(lb=theta_min, ub=theta_max)
bd

Now we will maximize the likelihood distribution, **the next step may take a few minutes!**

In [ ]:
from scipy.optimize import minimize # Minimization of scalar function

mu_data = np.array(df['MU_SH0ES']) # Load the experimental data for distance modulus
cov = cov_matrix

neg_like = lambda *args: -log_likelihood(*args) # Define the negative likelihood function: minimze negative log likelihood = maximize log likelihood

# Optimize the negative likelihood
soln = minimize(neg_like, initial, args=(mu_data, cov), method='Nelder-Mead', bounds=bd)
Omega_Lambda_ml, Omega_m_ml, h_ml = soln.x # Store the parameters that optimizes the likelihood

print("Optimized likelihood estimates:")
print("Omega_Lambda = {:.2f}".format(Omega_Lambda_ml))
print("Omega_m = {:.2f}".format(Omega_m_ml))
print("H0[Km/sMpc] = {:.2f}".format(h_ml*100)) # Print H0 in km/sMpc

Although the $\Omega_\Lambda$ and $\Omega_m$ didn't summed to 1, this constraint will be included in the total probability. 

In [ ]:
# Plot the mu_theoretical calculated using the parameters that maximize the likelihood
plt.figure(figsize=(15,7))

# Plot the error bars
x_exp=df['zCMB']
y_exp=df['MU_SH0ES']
plt.errorbar(x_exp, y_exp, xerr=df['zCMBERR'], yerr=df['MU_SH0ES_ERR_DIAG'], fmt=",c", ecolor="y", capsize=0)

# Plot the theoretical model with guess parameters
mu_guess=mu_model_pipe(Omega_Lambda_guess, Omega_m_guess, h_guess)
plt.plot(z_var, mu_guess(x_exp), "r", alpha=0.3, lw=3, label="mu_guess")

# Plot the theoretical model with optimized parameters
mu_opt = mu_model_pipe(Omega_Lambda_ml, Omega_m_ml, h_ml)
plt.plot(z_var, mu_opt(z_var), "b", alpha=0.3, lw=3, label="mu_opt")
plt.legend(fontsize=14)
plt.xlabel("zCMB")
plt.ylabel("Distance Modulus")

# Posterior Probability Function

In this section we will sample the posterior probability using the algorithm EMCEE to implement the Markov Chain Monte Carlo method. According to Bayes Theorem, the posterior probability is given by:
$$  p(\theta|\vec{x}) \propto p(\theta)\mathcal{L}(\vec{x}|\theta)\,, $$

where $\theta$ is the cosmological parameters vector, $\vec{x}$ the input data and $\mathcal{L}(\vec{x}|\theta)$ is the likelihood function.

In [ ]:
# Defining the prior probability function

def log_prior(theta):
    Omega_Lambda, Omega_m, h = theta # Initial parameters guess
    if 0.67 < Omega_Lambda < 0.71 and 0.29 < Omega_m < 0.33 and 0.65 < h < 0.69 and Omega_m+Omega_Lambda==1:
        return 0.0
    return -np.inf

In [ ]:
# Defining the full log-probability function

def log_probability(theta, mu_data, cov):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, mu_data, cov)

We’ll start by initializing the walkers in a tiny Gaussian ball around the maximum likelihood result and then run 10,000 steps of MCMC using 2000 walkers.

In [ ]:
# Define the initial position
pos = soln.x + 10**(-1)*np.random.randn(2000, 3)
nwalkers, ndim = pos.shape
pos

In [ ]:
# Generating samples with emcee
import emcee

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(mu_data, cov))
sampler.run_mcmc(pos, 10000, progress=True);

# Plotting the Parameters Estimatitive

About 40 steps are needed for the chain to “forget” where it started. Let’s discard the initial 100 steps, thin by about 15 steps, and flatten the chain so that we have a flat list of samples:

In [ ]:
# get_chain attribute from EnsembleSampler emcee:
# flat – Flatten the chain across the ensemble. (default: False)
# thin – Take only every thin steps from the chain. (default: 1)
# discard – Discard the first discard steps in the chain as burn-in. (default: 0)
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
labels = ["\Omega_\Lambda", "\Omega_m", "h"]

In [ ]:
import corner

fig = corner.corner(
    flat_samples, labels=labels, truths=[Omega_Lambda_guess, Omega_m_guess, h_guess]
)
plt.savefig('corner.png');

The corner plot shows the marginalized distribution for each parameter independently in the histograms along the diagonal and then the marginalized two dimensional distributions in the other panels.

Finally, we will compute the final parameter estimations as the 50-th percentile of the sample and the uncertainties as the difference between the 25-th,  75-th percentile and the 50-th percentile of the samples:

In [ ]:
#Given a vector V of length N, the q-th percentile of V is the value q/100 of the way from the minimum to the maximum in a sorted copy of V
aux = []
for i in range(ndim):
    aux.append(np.percentile(flat_samples[:, i], [25, 50, 75])) # Compute the 25-th, 50-th and 75-th percentile of the data along the specified axis.
aux

In [ ]:
# Store the percentiles in a array
mcmc = np.array([aux[0], aux[1], aux[2]*100])

In [ ]:
from IPython.display import display, Math

for i in range(ndim):
    q = np.diff(mcmc[i]) # Calculate the n-th discrete difference along the given axis: a[i+1] - a[i]
    labels2 = ["\Omega_\Lambda", "\Omega_m", "H_0[{\r km/sMpc}]"]
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{+{2:.3f}}}"
    txt = txt.format(mcmc[i][1], q[0], q[1], labels2[i])
    display(Math(txt))

**These are the final values estimated by the ensemble sampler emcee for the dark energy density $\Omega_\Lambda$, the mass density $\Omega_m$ and the Hubble parameter $H_0$.**

# Conclusion
We have analysed the supernova data from Phanteon+SH0ES collaboration to estimate the universe fractions of matter, vaccum energy and the the Hubble parameter using bayesian inference. These parameters was computed considering 10000 Markov Chains generated by 2000 walkers using the emcee implementation of the Monte Carlo algorithm. Since about 40 steps are needed for the chain to "forget" where it started we discarded the initial 100 steps. The final estimation choosen was the 50-th percentile of the flatted samples and the uncertainty the diference of the central value with the 25-th and the 75-th percentiles respectively. The final values encountered are matter density 
$\Omega_m = 0.333^{+0.066}_{−0.068}$, the dark energy density $\Omega_\Lambda = 0.711^{+0.071}_{−0.066}$ and the Hubble parameter $H_0 = 68.712^{+6.552}_{−6.497}{\rm km/s/Mpc}$ which agree with the direct observation of these parameters released by th Planck Satelite collaboration (2018): $H_0 = (67.4\pm0.5){\rm km/s/Mpc};  \Omega_m=0.315\pm0.007$ and $\Omega_\Lambda = 1 - \Omega_m$.

# References
* [Phanteon+SH0ES results 2021/2022](https://pantheonplussh0es.github.io/)
* [Library emcee documentation](https://emcee.readthedocs.io/en/stable/tutorials/line/)
* [Planck Satelite collaboration (2018)](https://arxiv.org/pdf/1807.06209.pdf)